In [1]:
## all imports
from IPython.display import HTML
import numpy as np
from urllib.request import urlopen  #html = urlopen("http://www.google.com/")
import bs4 #this is beautiful soup
import time
import operator
import socket
#import cPickle
try:
    import pickle
except ImportError:
    import cPickle as pickle
import re # regular expressions

from pandas import Series
import pandas as pd
from pandas import DataFrame

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set_context("talk")
sns.set_style("white")

from secret import *

C:\Anaconda3\lib\site-packages\matplotlib\__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [2]:
a = np.zeros(3)
#a.clip? --> help

In [3]:
#read columns into data frame
u_cols = ['user_id','age', 'sex', 'occupation','zip_code']
users = pd.read_csv('http://files.grouplens.org/datasets/movielens/ml-100k/u.user', 
    sep='|', names=u_cols)

users.head()

,user_id,age,sex,occupation,zip_code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


In [4]:
r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
ratings = pd.read_csv(
    'http://files.grouplens.org/datasets/movielens/ml-100k/u.data', 
    sep='\t', names=r_cols)

ratings.head() 

,user_id,movie_id,rating,unix_timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [5]:
# the movies file contains columns indicating the movie's genres
# let's only load the first five columns of the file with usecols
m_cols = ['movie_id', 'title', 'release_date', 
            'video_release_date', 'imdb_url']

movies = pd.read_csv(
    'http://files.grouplens.org/datasets/movielens/ml-100k/u.item', 
    sep='|', names=m_cols, usecols=range(5))

movies.head()

,movie_id,title,release_date,video_release_date,imdb_url
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995)


In [6]:
print(movies.dtypes)
print()
print(movies.describe())

movie_id                int64
title                  object
release_date           object
video_release_date    float64
imdb_url               object
dtype: object

          movie_id  video_release_date
count  1682.000000                   0
mean    841.500000                 NaN
std     485.695893                 NaN
min       1.000000                 NaN
25%     421.250000                 NaN
50%     841.500000                 NaN
75%    1261.750000                 NaN
max    1682.000000                 NaN


In [7]:
print(users.occupation.head())

users['occupation'].head()

0    technician
1         other
2        writer
3    technician
4         other
Name: occupation, dtype: object


0    technician
1         other
2        writer
3    technician
4         other
Name: occupation, dtype: object

In [8]:
columns_you_want = ['occupation','sex']
print(users[columns_you_want].head())

#print(users.iloc[3])  #printing 3rd row

   occupation sex
0  technician   M
1       other   F
2      writer   M
3  technician   M
4       other   F


# Filtering
## old users greater than 25 years

In [9]:
#users greater than 40 and male
users[(users.age==40) & (users.sex=='M')].head()

,user_id,age,sex,occupation,zip_code
18,19,40,M,librarian,02138
82,83,40,M,other,44133
115,116,40,M,healthcare,97232
199,200,40,M,programmer,93402
283,284,40,M,executive,92629


In [10]:
## users who are female and programmers
selected_users = users[(users.sex == 'F') & 
                       (users.occupation == 'programmer')]

## show statistic summary
print(selected_users.describe())

## alternatives:
print(selected_users.age.mean())
print(selected_users['age'].mean())

          user_id        age
count    6.000000   6.000000
mean   411.166667  32.166667
std    149.987222   5.115336
min    292.000000  26.000000
25%    313.000000  28.250000
50%    378.000000  32.000000
75%    416.750000  36.500000
max    698.000000  38.000000
32.166666666666664
32.166666666666664


# Split Apply and Combine
- splitting the data into groups based on some criteria
- applying a function to each group independently
- combining the results into a data structure

# Find Diligent Users
- split data per user ID
- count ratings
- combine result

In [11]:
print(ratings.head())

#splitting
#grouped_data = ratings.groupby('user_id')
grouped_data = ratings['movie_id'].groupby(ratings['user_id'])

## count and combine
ratings_per_user = grouped_data.count()

print(ratings_per_user.tail(5))


   user_id  movie_id  rating  unix_timestamp
0      196       242       3       881250949
1      186       302       3       891717742
2       22       377       1       878887116
3      244        51       2       880606923
4      166       346       1       886397596
user_id
939     49
940    107
941     22
942     79
943    168
dtype: int64


# Quiz
- Average rating per movie
- movie titles with the highest average rating

In [15]:
#Split the data
grouped_data = ratings['rating'].groupby(ratings['movie_id'])

##average and combine
average_rating = grouped_data.mean()
print("Average ratingss: ")
print(average_rating.head())

print()

maximum_rating = average_rating.max()
good_movie_ids = average_rating[average_rating == maximum_rating].index


print()

print("Best movie titles: ")
print(movies[movies.movie_id.isin(good_movie_ids)].title)

Average ratingss: 
movie_id
1    3.878319
2    3.206107
3    3.033333
4    3.550239
5    3.302326
Name: rating, dtype: float64


Best movie titles: 
813                         Great Day in Harlem, A (1994)
1121                       They Made Me a Criminal (1939)
1188                                   Prefontaine (1997)
1200           Marlene Dietrich: Shadow and Light (1996) 
1292                                      Star Kid (1997)
1466                 Saint of Fort Washington, The (1993)
1499                            Santa with Muscles (1996)
1535                                 Aiqing wansui (1994)
1598                        Someone Else's America (1995)
1652    Entertaining Angels: The Dorothy Day Story (1996)
Name: title, dtype: object


In [16]:
#how many ratings per movie
how_many_ratings = grouped_data.count()

print("Number of ratings per movie: ")
print(how_many_ratings[average_rating == maximum_rating])


Number of ratings per movie: 
movie_id
814     1
1122    1
1189    3
1201    1
1293    3
1467    2
1500    2
1536    1
1599    1
1653    1
dtype: int64


# Passing a function

In [17]:
average_rating = grouped_data.apply(lambda f: f.mean())
average_rating.head()

movie_id
1    3.878319
2    3.206107
3    3.033333
4    3.550239
5    3.302326
dtype: float64

# Quiz
- get the average rating per user
- advanced: list all occupations and if they are male or female dominant

In [18]:
grouped_data = ratings['rating'].groupby(ratings['user_id'])
average_rating = grouped_data.mean()
average_rating.head()

user_id
1    3.610294
2    3.709677
3    2.796296
4    4.333333
5    2.874286
Name: rating, dtype: float64

In [22]:
grouped_data = users['sex'].groupby(users['occupation'])
male_dominant_occupations = grouped_data.apply(lambda f: sum(f=='M') > sum(f=='F'))
print(male_dominant_occupations)
#print('\n')

occupation
administrator     True
artist            True
doctor            True
educator          True
engineer          True
entertainment     True
executive         True
healthcare       False
homemaker        False
lawyer            True
librarian        False
marketing         True
none              True
other             True
programmer        True
retired           True
salesman          True
scientist         True
student           True
technician        True
writer            True
dtype: bool


In [25]:
print("Number of male users: ")
print(sum(users['sex']=='M'))

print("Number of female users: ")
print(sum(users['sex']=='F'))

Number of male users: 
670
Number of female users: 
273


# Pandas "wrapup"
- create data frames
- get sub-frames
- filter data
- use group-by
- apply a user defined function

# Python data scraping¶
## Why scrape the web?
- vast source of information
- automate tasks
- keep up with sites
- fun!
## Can you think of examples ?
Answer: Some examples we had were stock market monitoring, sports data, or airline prices.

In [52]:
url = 'http://www.crummy.com/software/BeautifulSoup'
source = urlopen(url).read()
print(source)


b'<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.0 Transitional//EN"\n"http://www.w3.org/TR/REC-html40/transitional.dtd">\n<html>\n<head>\n<meta http-equiv="Content-Type" content="text/html; charset=utf-8">\n<title>Beautiful Soup: We called him Tortoise because he taught us.</title>\n<link rev="made" href="mailto:leonardr@segfault.org">\n<link rel="stylesheet" type="text/css" href="/nb/themes/Default/nb.css">\n<meta name="Description" content="Beautiful Soup: a library designed for screen-scraping HTML and XML.">\n<meta name="generator" content="Markov Approximation 1.4 (module: leonardr)">\n<meta name="author" content="Leonard Richardson">\n</head>\n<body bgcolor="white" text="black" link="blue" vlink="660066" alink="red">\n<img align="right" src="10.1.jpg" width="250"><br />\n\n<p>You didn\'t write that awful page. You\'re just trying to get some\ndata out of it. Beautiful Soup is here to help. Since 2004, it\'s been\nsaving programmers hours or days of work on quick-turnaround\nscreen sc

# Quiz :
- Is the word 'Alice' mentioned on the beautiful soup homepage?
- How often does the word 'Soup' occur on the site?
- - hint: use .count()
- At what index occurs the substring 'alien video games' ?
- - hint: use .find()

In [53]:
## is Alice in source?  ##we encode strings into bytes for python 3
print('Alice'.encode() in source)

##count occurences in soup
print(source.count('Soup'.encode()))

##find index of Alien video games
position = source.find('alien video games'.encode())
print(position)

## quick test to see the substring in the source variable
## you can access strings like lists
print(source[position:position + 20])

## or the tidier version:
print(source[position:position + len('alien video games'.encode())])

False
41
1639
b'alien video games</a'
b'alien video games'


# Beautiful Soup
- designed to make your life easier
- many good functions for parsing html code
# Some examples

In [58]:
#get bs4 object
soup = bs4.BeautifulSoup(source)
'''
print("------soup------")
print(soup)
print('\n')
print("------soup prettify-----------")
print(soup.prettify()) '''

'\nprint("------soup------")\nprint(soup)\nprint(\'\n\')\nprint("------soup prettify-----------")\nprint(soup.prettify()) '

In [62]:
# find all a tags
soup.findAll('a')

[<a href="bs4/download/"><h1>Beautiful Soup</h1></a>,
 <a href="#Download">Download</a>,
 <a href="bs4/doc/">Documentation</a>,
 <a href="#HallOfFame">Hall of Fame</a>,
 <a href="https://code.launchpad.net/beautifulsoup">Source</a>,
 <a href="https://groups.google.com/forum/?fromgroups#!forum/beautifulsoup">Discussion group</a>,
 <a href="http://www.candlemarkandgleam.com/shop/constellation-games/"><i>Constellation
 Games</i>, my sci-fi novel about alien video games</a>,
 <a href="http://constellation.crummy.com/Constellation%20Games%20excerpt.html">read
 the first two chapters for free</a>,
 <a href="https://groups.google.com/forum/?fromgroups#!forum/beautifulsoup">the discussion
 group</a>,
 <a href="https://bugs.launchpad.net/beautifulsoup/">file it</a>,
 <a href="http://lxml.de/">lxml</a>,
 <a href="http://code.google.com/p/html5lib/">html5lib</a>,
 <a href="bs4/doc/">Read more.</a>,
 <a name="Download"><h2>Download Beautiful Soup</h2></a>,
 <a href="bs4/download/">Beautiful Soup
 

The last command only returns an empty list, because Soup is not an HTML tag. It is just a string that occours in the webpage.
# Some examples

In [64]:
## get attribute value from an element:
## find tag: this only returns the first occurrence, not all tags in the string
first_tag = soup.find('a')

print("----")
## getting attribute href
first_tag.get('href')
print("----")
## get all links on the page
link_list = [l.get('href') for l in soup.findAll('a')]
link_list

----
----


['bs4/download/',
 '#Download',
 'bs4/doc/',
 '#HallOfFame',
 'https://code.launchpad.net/beautifulsoup',
 'https://groups.google.com/forum/?fromgroups#!forum/beautifulsoup',
 'http://www.candlemarkandgleam.com/shop/constellation-games/',
 'http://constellation.crummy.com/Constellation%20Games%20excerpt.html',
 'https://groups.google.com/forum/?fromgroups#!forum/beautifulsoup',
 'https://bugs.launchpad.net/beautifulsoup/',
 'http://lxml.de/',
 'http://code.google.com/p/html5lib/',
 'bs4/doc/',
 None,
 'bs4/download/',
 'http://www.crummy.com/software/BeautifulSoup/bs3/documentation.html',
 'download/3.x/BeautifulSoup-3.2.1.tar.gz',
 None,
 'http://www.nytimes.com/2007/10/25/arts/design/25vide.html',
 'https://github.com/reddit/reddit/blob/85f9cff3e2ab9bb8f19b96acd8da4ebacc079f04/r2/r2/lib/media.py',
 'http://www.harrowell.org.uk/viktormap.html',
 'http://svn.python.org/view/tracker/importer/',
 'http://www2.ljworld.com/',
 'http://www.b-list.org/weblog/2010/nov/02/news-done-broke/',
 '

In [65]:
## filter all external links
# create an empty list to collect the valid links
external_links = []

# write a loop to filter the links
# if it starts with 'http' we are happy
for l in link_list:
    if l[:4]=="http":
        external_links.append(l)

TypeError: 'NoneType' object is not subscriptable

In [66]:
# lets investigate. Have a close look at the link_list:
link_list

# Seems that there are None elements!
# Let's verify
#print sum([l is None for l in link_list])

# So there are two elements in the list that are None!

['bs4/download/',
 '#Download',
 'bs4/doc/',
 '#HallOfFame',
 'https://code.launchpad.net/beautifulsoup',
 'https://groups.google.com/forum/?fromgroups#!forum/beautifulsoup',
 'http://www.candlemarkandgleam.com/shop/constellation-games/',
 'http://constellation.crummy.com/Constellation%20Games%20excerpt.html',
 'https://groups.google.com/forum/?fromgroups#!forum/beautifulsoup',
 'https://bugs.launchpad.net/beautifulsoup/',
 'http://lxml.de/',
 'http://code.google.com/p/html5lib/',
 'bs4/doc/',
 None,
 'bs4/download/',
 'http://www.crummy.com/software/BeautifulSoup/bs3/documentation.html',
 'download/3.x/BeautifulSoup-3.2.1.tar.gz',
 None,
 'http://www.nytimes.com/2007/10/25/arts/design/25vide.html',
 'https://github.com/reddit/reddit/blob/85f9cff3e2ab9bb8f19b96acd8da4ebacc079f04/r2/r2/lib/media.py',
 'http://www.harrowell.org.uk/viktormap.html',
 'http://svn.python.org/view/tracker/importer/',
 'http://www2.ljworld.com/',
 'http://www.b-list.org/weblog/2010/nov/02/news-done-broke/',
 '

In [67]:
# Let's filter those objects out in the for loop
external_links = []

# write a loop to filter the links
# if it is not None and starts with 'http' we are happy
for l in link_list:
    if l is not None and l[:4]=='http':
        external_links.append(l)
external_links

['https://code.launchpad.net/beautifulsoup',
 'https://groups.google.com/forum/?fromgroups#!forum/beautifulsoup',
 'http://www.candlemarkandgleam.com/shop/constellation-games/',
 'http://constellation.crummy.com/Constellation%20Games%20excerpt.html',
 'https://groups.google.com/forum/?fromgroups#!forum/beautifulsoup',
 'https://bugs.launchpad.net/beautifulsoup/',
 'http://lxml.de/',
 'http://code.google.com/p/html5lib/',
 'http://www.crummy.com/software/BeautifulSoup/bs3/documentation.html',
 'http://www.nytimes.com/2007/10/25/arts/design/25vide.html',
 'https://github.com/reddit/reddit/blob/85f9cff3e2ab9bb8f19b96acd8da4ebacc079f04/r2/r2/lib/media.py',
 'http://www.harrowell.org.uk/viktormap.html',
 'http://svn.python.org/view/tracker/importer/',
 'http://www2.ljworld.com/',
 'http://www.b-list.org/weblog/2010/nov/02/news-done-broke/',
 'http://esrl.noaa.gov/gsd/fab/',
 'http://laps.noaa.gov/topograbber/',
 'http://groups.google.com/group/beautifulsoup/',
 'https://launchpad.net/beauti

In [68]:
# and we can put this in a list comprehension as well, it almost reads like 
# a sentence.

## [l for l in link_list if l is not None and l.startswith('http')]

# Parsing the Tree

In [69]:
# redifining `s` without any line breaks
s = """<!DOCTYPE html><html><head><title>This is a title</title></head><body><h3> Test </h3><p>Hello world!</p></body></html>"""
## get bs4 object
tree = bs4.BeautifulSoup(s)

## get html root node
root_node = tree.html

##get head from root using contents
head = root_node.contents[0]

##get body from root
body = root_node.contents[1]

## could directly access body
tree.body

<body><h3> Test </h3><p>Hello world!</p></body>

# Quiz:
- Find the h3 tag by parsing the tree starting at body
- Create a list of all Hall of Fame entries listed on the Beautiful Soup webpage
- . hint: it is the only unordered list in the page (tag ul)

In [70]:
#get html 3 tag from body
body.contents[0]

<h3> Test </h3>

In [74]:
# use 'ul' as entry point
entry_point = soup.find('ul'.encode())
## get hall of fame list from entry point
## skip the first entry 
hall_of_fame_list = entry_point.contents[1:]

# reformat string into a list containing strings
tmp=[]
for li in hall_of_fame_list:
    tmp.append(li.contents)
tmp

[[<a href="http://www.nytimes.com/2007/10/25/arts/design/25vide.html">"Movable
   Type"</a>,
  ', a work of digital art on display in the lobby of the New\n York Times building, uses Beautiful Soup to scrape news feeds.\n\n'],
 ['Reddit uses Beautiful Soup to ',
  <a href="https://github.com/reddit/reddit/blob/85f9cff3e2ab9bb8f19b96acd8da4ebacc079f04/r2/r2/lib/media.py">parse
  a page that's been linked to and find a representative image</a>,
  '.\n\n'],
 ['Alexander Harrowell uses Beautiful Soup to ',
  <a href="http://www.harrowell.org.uk/viktormap.html">track the business
   activities</a>,
  ' of an arms merchant.\n\n'],
 ['The developers of Python itself used Beautiful Soup to ',
  <a href="http://svn.python.org/view/tracker/importer/">migrate the Python
  bug tracker from Sourceforge to Roundup</a>,
  '.\n\n'],
 ['The ',
  <a href="http://www2.ljworld.com/">Lawrence Journal-World</a>,
  '\nuses Beautiful Soup to ',
  <a href="http://www.b-list.org/weblog/2010/nov/02/news-done-bro

tmp now is actually a list of lists containing the hall of fame entries. Here is some advanced Python on how to print really just one entry per list item.
The cool things about this are:
- The use of "" to just access the join function of strings.
- The join function itself
- that you can actually have two nested for loops in a list comprehension

In [75]:
test = ["".join(str(a) for a in sublist) for sublist in tmp]
print('\n'.join(test))

<a href="http://www.nytimes.com/2007/10/25/arts/design/25vide.html">"Movable
 Type"</a>, a work of digital art on display in the lobby of the New
 York Times building, uses Beautiful Soup to scrape news feeds.


Reddit uses Beautiful Soup to <a href="https://github.com/reddit/reddit/blob/85f9cff3e2ab9bb8f19b96acd8da4ebacc079f04/r2/r2/lib/media.py">parse
a page that's been linked to and find a representative image</a>.


Alexander Harrowell uses Beautiful Soup to <a href="http://www.harrowell.org.uk/viktormap.html">track the business
 activities</a> of an arms merchant.


The developers of Python itself used Beautiful Soup to <a href="http://svn.python.org/view/tracker/importer/">migrate the Python
bug tracker from Sourceforge to Roundup</a>.


The <a href="http://www2.ljworld.com/">Lawrence Journal-World</a>
uses Beautiful Soup to <a href="http://www.b-list.org/weblog/2010/nov/02/news-done-broke/">gather
statewide election results</a>.


The <a href="http://esrl.noaa.gov/gsd/fab/">NOAA

# Advanced Example
## Idea by Jesse Steinweg-Woods
# Scraping data science skills¶
- What skills are in demand for data scientists?
- Should we have a lecture on Spark or only on MapReduce?
We want to scrape the information from job advertisements for data scientists from indeed.com Let's scrape and find out!

In [77]:
# Fixed url for job postings containing data scientist
url = 'http://www.indeed.com/jobs?q=data+scientist&l='
# read the website
source = urlopen(url).read()
# parse html code
bs_tree = bs4.BeautifulSoup(source)

In [81]:
#see how many job postings we found
job_count_string = bs_tree.find(id = 'searchCount').contents[0]
job_count_string = job_count_string.split()[-1]
print('Search yielded %s hits. ' %(job_count_string))

Search yielded 21,554 hits. 


In [89]:
# not that job_count so far is still a string, 
# not an integer, and the , separator prevents 
# us from just casting it to int
job_count_digits = [int(d) for d in job_count_string if d.isdigit()]
print(job_count_digits)

##convert these digits to a number
digits = ''.join(str(n) for n in job_count_digits)
int(digits)

[2, 1, 5, 5, 4]


21554